In [1]:
!nvidia-smi


Mon Dec 29 09:48:34 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [15]:
%%writefile main.cu
// ==============================
// 0) Подключаем библиотеки
// ==============================

#include <cuda_runtime.h>              // CUDA: память + запуск kernel
#include <iostream>                    // cout/cerr
#include <vector>                      // std::vector
#include <random>                      // генерация случайных чисел
#include <chrono>                      // тайминг CPU
#include <climits>                     // INT_MAX
#include <algorithm>                   // std::swap

// ==============================
// 0.1) Макрос проверки CUDA-ошибок
// ==============================

// CHECK(...) — если CUDA-вызов провалился, выводим ошибку и выходим
#define CHECK(x) do { \
  cudaError_t err = (x); \
  if (err != cudaSuccess) { \
    std::cerr << "CUDA error: " << cudaGetErrorString(err) \
              << " at " << __FILE__ << ":" << __LINE__ << "\n"; \
    std::exit(1); \
  } \
} while(0)


// ============================================================
// 1) CPU BLOCK: последовательные сортировки + тайминг
// ============================================================

// ------------------------------
// 1.1) CPU MergeSort
// ------------------------------

// Сливаем две отсортированные части: [l..m) и [m..r)
static void cpu_merge(std::vector<int>& a, int l, int m, int r, std::vector<int>& tmp) {
  int i = l;                              // индекс в левой части
  int j = m;                              // индекс в правой части
  int k = l;                              // индекс записи во временный массив

  while (i < m && j < r) {                // пока обе части не закончились
    tmp[k++] = (a[i] <= a[j]) ? a[i++] : a[j++]; // берём меньший элемент
  }

  while (i < m) tmp[k++] = a[i++];        // дописываем остаток левой части
  while (j < r) tmp[k++] = a[j++];        // дописываем остаток правой части

  for (int t = l; t < r; t++) a[t] = tmp[t]; // копируем обратно в исходный массив
}

// Рекурсивная часть MergeSort
static void cpu_merge_sort_rec(std::vector<int>& a, int l, int r, std::vector<int>& tmp) {
  if (r - l <= 1) return;                 // 0 или 1 элемент уже отсортирован

  int m = l + (r - l) / 2;                // середина диапазона
  cpu_merge_sort_rec(a, l, m, tmp);        // сортируем левую часть
  cpu_merge_sort_rec(a, m, r, tmp);        // сортируем правую часть
  cpu_merge(a, l, m, r, tmp);              // сливаем
}

// Внешняя функция CPU MergeSort
static void cpu_merge_sort(std::vector<int>& a) {
  std::vector<int> tmp(a.size());         // временный массив
  cpu_merge_sort_rec(a, 0, (int)a.size(), tmp); // запускаем рекурсию
}

// ------------------------------
// 1.2) CPU QuickSort
// ------------------------------

// Разделение массива по pivot (опорному элементу)
static int cpu_partition(std::vector<int>& a, int l, int r) {
  int pivot = a[r - 1];                   // pivot = последний элемент
  int i = l;                              // индекс позиции для меньших элементов

  for (int j = l; j < r - 1; j++) {       // пробегаем все элементы, кроме pivot
    if (a[j] <= pivot) {                  // если элемент <= pivot
      std::swap(a[i], a[j]);              // меняем местами
      i++;                                // увеличиваем границу
    }
  }

  std::swap(a[i], a[r - 1]);              // ставим pivot на правильное место
  return i;                               // возвращаем индекс pivot
}

// Рекурсивный CPU QuickSort
static void cpu_quick_sort_rec(std::vector<int>& a, int l, int r) {
  if (r - l <= 1) return;                 // база рекурсии

  int p = cpu_partition(a, l, r);         // делим массив на части
  cpu_quick_sort_rec(a, l, p);            // сортируем левую часть
  cpu_quick_sort_rec(a, p + 1, r);        // сортируем правую часть
}

// Внешняя функция CPU QuickSort
static void cpu_quick_sort(std::vector<int>& a) {
  cpu_quick_sort_rec(a, 0, (int)a.size()); // запуск
}

// ------------------------------
// 1.3) CPU HeapSort
// ------------------------------

// Функция "просеивания" вниз для кучи
static void cpu_heapify(std::vector<int>& a, int n, int i) {
  int largest = i;                        // считаем i самым большим
  int L = 2 * i + 1;                      // левый ребёнок
  int R = 2 * i + 2;                      // правый ребёнок

  if (L < n && a[L] > a[largest]) largest = L; // выбираем большего
  if (R < n && a[R] > a[largest]) largest = R; // выбираем большего

  if (largest != i) {                     // если ребёнок больше
    std::swap(a[i], a[largest]);          // меняем
    cpu_heapify(a, n, largest);           // продолжаем вниз
  }
}

// CPU HeapSort
static void cpu_heap_sort(std::vector<int>& a) {
  int n = (int)a.size();                  // размер массива

  for (int i = n / 2 - 1; i >= 0; i--) {  // строим кучу снизу вверх
    cpu_heapify(a, n, i);                 // heapify
  }

  for (int i = n - 1; i > 0; i--) {       // извлекаем максимум
    std::swap(a[0], a[i]);                // перемещаем максимум в конец
    cpu_heapify(a, i, 0);                 // восстанавливаем кучу
  }
}

// ------------------------------
// 1.4) Функция тайминга CPU (мс)
// ------------------------------
static double cpu_ms(std::chrono::high_resolution_clock::time_point a,
                     std::chrono::high_resolution_clock::time_point b) {
  return std::chrono::duration<double, std::milli>(b - a).count(); // ms
}


// ============================================================
// 2) GPU BLOCK: сортировка чанков + простое merge + тайминг
// ============================================================

// ------------------------------
// 2.1) GPU MergeSort: bitonic сортировка чанка (параллельно)
// ------------------------------

// CHUNK — размер чанка (и threads per block)
// каждый блок сортирует один чанк
template<int CHUNK>
__global__ void bitonicSortChunks(int* d, int n) {
  __shared__ int s[CHUNK];                // shared memory для чанка

  int chunkStart = blockIdx.x * CHUNK;    // начало чанка
  int tid = threadIdx.x;                  // номер потока в блоке
  int idx = chunkStart + tid;             // глобальный индекс

  s[tid] = (idx < n) ? d[idx] : INT_MAX;  // грузим элемент или INT_MAX
  __syncthreads();                        // ждём все потоки

  for (int k = 2; k <= CHUNK; k <<= 1) {  // размер "подпоследовательности"
    for (int j = k >> 1; j > 0; j >>= 1) {// шаг сравнения
      int ixj = tid ^ j;                  // "пара" для сравнения
      if (ixj > tid) {                    // чтобы не сравнивать дважды
        bool up = ((tid & k) == 0);       // направление сортировки
        int a = s[tid];                   // значение потока tid
        int b = s[ixj];                   // значение потока ixj
        if ((up && a > b) || (!up && a < b)) { // условие swap
          s[tid] = b;                     // меняем
          s[ixj] = a;                     // меняем
        }
      }
      __syncthreads();                    // синхронизация
    }
  }

  if (idx < n) d[idx] = s[tid];           // записываем обратно
}

// ------------------------------
// 2.2) GPU QuickSort: сортировка чанка одним потоком
// ------------------------------

// простая device-функция quicksort (итеративно) для массива в shared
__device__ void device_quick_sort(int* a, int n) {
  int L[64], R[64];                       // стек диапазонов
  int top = 0;                            // вершина стека
  L[top] = 0; R[top] = n - 1; top++;      // кладём первый диапазон

  while (top > 0) {                       // пока стек не пуст
    top--;                                // берём последний диапазон
    int l = L[top], r = R[top];           // границы
    if (l >= r) continue;                 // если 0/1 элемент — пропуск

    int pivot = a[r];                     // pivot
    int i = l;                            // индекс разделения
    for (int j = l; j < r; j++) {         // проход по диапазону
      if (a[j] <= pivot) {                // если <= pivot
        int t = a[i]; a[i] = a[j]; a[j] = t; // swap
        i++;                              // сдвиг границы
      }
    }
    { int t = a[i]; a[i] = a[r]; a[r] = t; } // ставим pivot на место

    if (i - 1 > l) { L[top] = l; R[top] = i - 1; top++; } // левый диапазон
    if (i + 1 < r) { L[top] = i + 1; R[top] = r; top++; } // правый диапазон
  }
}

// kernel: загружаем чанк в shared, поток 0 сортирует, остальные ждут
template<int CHUNK>
__global__ void quickSortChunks(int* d, int n) {
  __shared__ int s[CHUNK];                // shared memory для чанка

  int chunkStart = blockIdx.x * CHUNK;    // начало чанка
  int tid = threadIdx.x;                  // поток
  int idx = chunkStart + tid;             // глобальный индекс

  s[tid] = (idx < n) ? d[idx] : INT_MAX;  // грузим в shared
  __syncthreads();                        // ждём всех

  if (tid == 0) {                         // только один поток сортирует
    int len = min(CHUNK, n - chunkStart); // реальная длина чанка
    device_quick_sort(s, len);            // сортировка
  }
  __syncthreads();                        // ждём завершения сортировки

  if (idx < n) d[idx] = s[tid];           // записываем обратно
}

// ------------------------------
// 2.3) GPU HeapSort: сортировка чанка одним потоком
// ------------------------------

// heapify на device
__device__ void device_heapify(int* a, int n, int i) {
  while (true) {                          // цикл "вниз"
    int largest = i;                      // предполагаем i самый большой
    int L = 2 * i + 1;                    // левый ребёнок
    int R = 2 * i + 2;                    // правый ребёнок
    if (L < n && a[L] > a[largest]) largest = L; // выбираем большего
    if (R < n && a[R] > a[largest]) largest = R; // выбираем большего
    if (largest == i) break;              // если i уже на месте — выход
    int t = a[i]; a[i] = a[largest]; a[largest] = t; // swap
    i = largest;                          // спускаемся вниз
  }
}

// heapsort на device
__device__ void device_heap_sort(int* a, int n) {
  for (int i = n / 2 - 1; i >= 0; i--) device_heapify(a, n, i); // строим кучу
  for (int i = n - 1; i > 0; i--) {     // извлекаем максимум
    int t = a[0]; a[0] = a[i]; a[i] = t; // swap
    device_heapify(a, i, 0);            // heapify
  }
}

// kernel: поток 0 сортирует чанк heapsort
template<int CHUNK>
__global__ void heapSortChunks(int* d, int n) {
  __shared__ int s[CHUNK];                // shared memory

  int chunkStart = blockIdx.x * CHUNK;    // начало чанка
  int tid = threadIdx.x;                  // поток
  int idx = chunkStart + tid;             // глобальный индекс

  s[tid] = (idx < n) ? d[idx] : INT_MAX;  // грузим в shared
  __syncthreads();                        // синхронизация

  if (tid == 0) {                         // один поток сортирует
    int len = min(CHUNK, n - chunkStart); // реальная длина чанка
    device_heap_sort(s, len);             // heapsort
  }
  __syncthreads();                        // ждём завершения сортировки

  if (idx < n) d[idx] = s[tid];           // запись обратно
}

// ------------------------------
// 2.4) GPU merge: простое попарное слияние (без merge-path)
// ------------------------------

__global__ void mergePairsSimple(const int* in, int* out, int n, int run) {
  int pairId = blockIdx.x;                       // номер пары сегментов
  int left0 = pairId * (2 * run);                // начало пары
  if (left0 >= n) return;                        // если вышли за массив — выходим

  int mid0   = min(left0 + run, n);              // середина пары
  int right0 = min(left0 + 2 * run, n);          // конец пары

  const int* A = in + left0;                     // левый отсортированный сегмент
  const int* B = in + mid0;                      // правый отсортированный сегмент
  int aN = mid0 - left0;                         // длина левого сегмента
  int bN = right0 - mid0;                        // длина правого сегмента
  int total = aN + bN;                           // общая длина после слияния

  int* C = out + left0;                          // куда пишем результат

  // каждый поток делает несколько позиций k в результирующем массиве
  for (int k = threadIdx.x; k < total; k += blockDim.x) {

    // i = сколько элементов берём из A, j = сколько из B, и i + j = k
    int iMin = max(0, k - bN);
    int iMax = min(k, aN);

    // бинарный поиск по i
    while (iMin <= iMax) {
      int i = (iMin + iMax) >> 1;
      int j = k - i;

      int aLeft  = (i > 0)  ? A[i - 1] : INT_MIN;
      int aRight = (i < aN) ? A[i]     : INT_MAX;
      int bLeft  = (j > 0)  ? B[j - 1] : INT_MIN;
      int bRight = (j < bN) ? B[j]     : INT_MAX;

      // нашли правильное разбиение
      if (aLeft <= bRight && bLeft <= aRight) {
        C[k] = (aRight <= bRight) ? aRight : bRight;  // элемент на позиции k
        break;
      }

      // сдвигаем бинарный поиск
      if (aLeft > bRight) iMax = i - 1;
      else                iMin = i + 1;
    }
  }
}


// ------------------------------
// 2.5) GPU wrapper: sort chunks + merge stages + cudaEvent timing
// ------------------------------

// режим какой алгоритм сортировки чанков использовать
enum GpuMode { GPU_MERGE = 1, GPU_QUICK = 2, GPU_HEAP = 3 };

template<int CHUNK>
float gpu_sort_chunk_and_merge(int* d, int n, int mode) {
  int blocks = (n + CHUNK - 1) / CHUNK;   // количество чанков (блоков)

  cudaEvent_t s, e;                       // CUDA события для тайминга
  CHECK(cudaEventCreate(&s));             // создаём start event
  CHECK(cudaEventCreate(&e));             // создаём end event
  CHECK(cudaEventRecord(s));              // старт таймера

  // 1) сортировка чанков
  if (mode == GPU_MERGE) {
    bitonicSortChunks<CHUNK><<<blocks, CHUNK>>>(d, n); // битоник параллельно
  } else if (mode == GPU_QUICK) {
    quickSortChunks<CHUNK><<<blocks, CHUNK>>>(d, n);   // quick в чанке
  } else {
    heapSortChunks<CHUNK><<<blocks, CHUNK>>>(d, n);    // heap в чанке
  }
  CHECK(cudaGetLastError());             // проверяем запуск kernel

  // 2) temp-массив для merge
  int* temp = nullptr;                   // указатель на временный буфер
  CHECK(cudaMalloc(&temp, n * sizeof(int))); // выделяем на GPU

  int run = CHUNK;                        // текущая длина отсортированного блока
  int* in  = d;                           // откуда читаем
  int* out = temp;                        // куда пишем

  // 3) поэтапное слияние: CHUNK -> 2CHUNK -> 4CHUNK -> ...
  while (run < n) {
    int pairs = (n + (2 * run) - 1) / (2 * run); // число пар
    mergePairsSimple<<<pairs, 256>>>(in, out, n, run); // простой merge
    CHECK(cudaGetLastError());            // проверка kernel
    std::swap(in, out);                   // меняем буферы местами
    run *= 2;                             // увеличиваем размер блока
  }

  // если итог оказался в temp — копируем в d
  if (in != d) {
    CHECK(cudaMemcpy(d, in, n * sizeof(int), cudaMemcpyDeviceToDevice));
  }

  CHECK(cudaFree(temp));                  // освобождаем temp

  CHECK(cudaEventRecord(e));              // стоп таймера
  CHECK(cudaEventSynchronize(e));         // ждём завершения всех GPU задач

  float ms = 0.0f;                        // время в миллисекундах
  CHECK(cudaEventElapsedTime(&ms, s, e)); // считаем время
  CHECK(cudaEventDestroy(s));             // удаляем события
  CHECK(cudaEventDestroy(e));             // удаляем события

  return ms;                              // возвращаем ms
}


// ============================================================
// 3) BENCHMARK BLOCK: сравнение CPU vs GPU
// ============================================================

int main() {
  std::vector<int> sizes = {10000, 100000, 1000000}; // тестовые размеры

  std::mt19937 rng(42);                               // генератор
  std::uniform_int_distribution<int> dist(0, 1000000);// диапазон

  for (int N : sizes) {                               // перебираем размеры
    std::vector<int> base(N);                         // исходный массив

    for (int i = 0; i < N; i++) base[i] = dist(rng);  // заполняем числами

    // ---------- CPU: считаем время каждой сортировки ----------
    auto cpuMerge = base;                             // копия для merge
    auto t1 = std::chrono::high_resolution_clock::now();// start
    cpu_merge_sort(cpuMerge);                         // merge sort
    auto t2 = std::chrono::high_resolution_clock::now();// end
    double cpuMergeMs = cpu_ms(t1, t2);               // ms

    auto cpuQuick = base;                             // копия для quick
    t1 = std::chrono::high_resolution_clock::now();   // start
    cpu_quick_sort(cpuQuick);                         // quick sort
    t2 = std::chrono::high_resolution_clock::now();   // end
    double cpuQuickMs = cpu_ms(t1, t2);               // ms

    auto cpuHeap = base;                              // копия для heap
    t1 = std::chrono::high_resolution_clock::now();   // start
    cpu_heap_sort(cpuHeap);                           // heap sort
    t2 = std::chrono::high_resolution_clock::now();   // end
    double cpuHeapMs = cpu_ms(t1, t2);                // ms

    // ---------- GPU: выделяем память один раз ----------
    int* d = nullptr;                                 // указатель на GPU массив
    CHECK(cudaMalloc(&d, N * sizeof(int)));           // выделяем память

    // ---------- GPU MergeSort ----------
    auto gpuMerge = base;                             // копия для GPU
    CHECK(cudaMemcpy(d, gpuMerge.data(), N*sizeof(int), cudaMemcpyHostToDevice)); // H2D
    float gpuMergeMs = gpu_sort_chunk_and_merge<512>(d, N, GPU_MERGE); // запуск
    CHECK(cudaMemcpy(gpuMerge.data(), d, N*sizeof(int), cudaMemcpyDeviceToHost)); // D2H
    bool okMerge = (gpuMerge == cpuMerge);            // проверка корректности

    // ---------- GPU QuickSort ----------
    auto gpuQuick = base;                             // копия
    CHECK(cudaMemcpy(d, gpuQuick.data(), N*sizeof(int), cudaMemcpyHostToDevice)); // H2D
    float gpuQuickMs = gpu_sort_chunk_and_merge<512>(d, N, GPU_QUICK); // запуск
    CHECK(cudaMemcpy(gpuQuick.data(), d, N*sizeof(int), cudaMemcpyDeviceToHost)); // D2H
    bool okQuick = (gpuQuick == cpuMerge);            // сравниваем с эталоном cpuMerge

    // ---------- GPU HeapSort ----------
    auto gpuHeap = base;                              // копия
    CHECK(cudaMemcpy(d, gpuHeap.data(), N*sizeof(int), cudaMemcpyHostToDevice)); // H2D
    float gpuHeapMs = gpu_sort_chunk_and_merge<512>(d, N, GPU_HEAP); // запуск
    CHECK(cudaMemcpy(gpuHeap.data(), d, N*sizeof(int), cudaMemcpyDeviceToHost)); // D2H
    bool okHeap = (gpuHeap == cpuMerge);              // сравнение с эталоном

    CHECK(cudaFree(d));                               // освобождаем GPU память

    // ---------- Вывод результатов ----------
    std::cout << "\n==============================\n";   // разделитель
    std::cout << "N = " << N << "\n";                 // размер
    std::cout << "==============================\n";   // разделитель

    std::cout << "CPU MergeSort: " << cpuMergeMs << " ms\n"; // CPU merge время
    std::cout << "CPU QuickSort: " << cpuQuickMs << " ms\n"; // CPU quick время
    std::cout << "CPU HeapSort:  " << cpuHeapMs  << " ms\n"; // CPU heap время

    std::cout << "GPU MergeSort: " << gpuMergeMs << " ms | " << (okMerge ? "OK" : "ERROR") << "\n"; // GPU merge
    std::cout << "GPU QuickSort: " << gpuQuickMs << " ms | " << (okQuick ? "OK" : "ERROR") << "\n"; // GPU quick
    std::cout << "GPU HeapSort:  " << gpuHeapMs  << " ms | " << (okHeap  ? "OK" : "ERROR") << "\n"; // GPU heap

    // speedup (если GPU быстрее, будет > 1)
    if (gpuMergeMs > 0) std::cout << "Speedup Merge (CPU/GPU): " << (cpuMergeMs / gpuMergeMs) << "x\n";
    if (gpuQuickMs > 0) std::cout << "Speedup Quick (CPU/GPU): " << (cpuQuickMs / gpuQuickMs) << "x\n";
    if (gpuHeapMs  > 0) std::cout << "Speedup Heap  (CPU/GPU): " << (cpuHeapMs  / gpuHeapMs ) << "x\n";
  }

  return 0;                                            // конец программы
}


Overwriting main.cu


In [16]:
!nvcc -std=c++17 -O3 main.cu -o main -gencode arch=compute_75,code=sm_75


In [17]:
!./main


N = 10000
CPU MergeSort: 1.10619 ms
CPU QuickSort: 0.722627 ms
CPU HeapSort:  0.944553 ms
GPU MergeSort: 0.57328 ms | OK
GPU QuickSort: 1.0711 ms | OK
GPU HeapSort:  1.61709 ms | OK
Speedup Merge (CPU/GPU): 1.92958x
Speedup Quick (CPU/GPU): 0.674656x
Speedup Heap  (CPU/GPU): 0.584107x

N = 100000
CPU MergeSort: 10.9185 ms
CPU QuickSort: 7.692 ms
CPU HeapSort:  11.2952 ms
GPU MergeSort: 4.50768 ms | OK
GPU QuickSort: 6.23587 ms | OK
GPU HeapSort:  8.07011 ms | OK
Speedup Merge (CPU/GPU): 2.4222x
Speedup Quick (CPU/GPU): 1.23351x
Speedup Heap  (CPU/GPU): 1.39964x

N = 1000000
CPU MergeSort: 134.164 ms
CPU QuickSort: 95.1539 ms
CPU HeapSort:  152.514 ms
GPU MergeSort: 57.7844 ms | OK
GPU QuickSort: 67.5834 ms | OK
GPU HeapSort:  74.0383 ms | OK
Speedup Merge (CPU/GPU): 2.32181x
Speedup Quick (CPU/GPU): 1.40795x
Speedup Heap  (CPU/GPU): 2.05993x
